# Introduction

QA plots for the TPC detector

In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [2]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.jp-CodeMirrorEditor').hide(); // for refer versions of nbviewer.org
 } else {
 $('div.input').show();
 $('div.jp-CodeMirrorEditor').show(); // for refer versions of nbviewer.org
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

*For the result HTML page:* 

In [3]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [4]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

via sPHENIX software distribution at `/cvmfs/sphenix.sdcc.bnl.gov/alma9.2-gcc-14.2.0/release/release_new/new.13`

## Plotting source code

In [5]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

View the source code repository at https://github.com/sPHENIX-Collaboration/QA-gallery.git

## JenkinsCI information (if available)

In [6]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))

Some further details about the QA run, if executed under the Jenkins CI:

* Link to the build: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-reconstruction-prdf/935/

* Git repo for macros: https://github.com/sPHENIX-Collaboration/macros.git , which merges `*/master` and the QA tracking branch

* Download the QA ROOT files: https://web.sdcc.bnl.gov/jenkins-sphenix/job/sPHENIX/job/test-tracking-reconstruction-prdf/935/display/redirect?page=artifacts

Automatically generated by [sPHENIX Jenkins continuous integration](https://web.sdcc.bnl.gov/jenkins-sphenix/) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)


# Initialization

In [7]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [8]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [9]:
%jsroot on

## Inputs and file checks

In [10]:
qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))

qa_file_name_ref = os.getenv("qa_file_name_ref")
if qa_file_name_ref is None:
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` env not set. use the default `qa_file_name_ref={qa_file_name_ref}`"))
elif qa_file_name_ref == 'None':
    qa_file_name_ref = None
    
    qa_file_name_ref = "prdf_reconstruction53877_qa.root"
    display(Markdown(f"`qa_file_name_ref` = None and we are set to not to use the reference histograms"))
    

display(Markdown(f"Opening QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

if qa_file_name_ref is not None:
#     qa_file_ref = ROOT.TFile.Open(qa_file_name_ref);

#     assert qa_file_ref.IsOpen()
    display(Markdown(f"Opening QA reference file at `{qa_file_name_ref}`"))
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = new TFile(\"{qa_file_name_ref}\");")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = \"{qa_file_name_ref}\";")
else:
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = nullptr;")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = nullptr;")

Opening QA file at `/var/lib/jenkins/workspace/sPHENIX/test-tracking-reconstruction-prdf@2/macros/TrackingProduction/prdf_reconstruction53877_qa.root`

Opening QA reference file at `reference/prdf_reconstruction53877_qa.root`

## Raw Hit quantities

##### Pick 3 sectors and 3 regions to compare ADC distributions to

In [11]:
%%cpp


if(qa_file_new == nullptr)
{
    std::cout << "Failed to open file " << qa_file_name_new << std::endl;
    exit(1);
}

    gStyle->SetPalette(kRainBow);

    TCanvas* m_canvas = new TCanvas("m_canvas", "INTT Hits Summary", 800, 600);
    m_canvas->Divide(3, 3);

    // Get histograms
    TH1F *sec1_r0_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec1_R0");
    TH1F *sec1_r1_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec1_R1");
    TH1F *sec1_r2_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec1_R2");
    TH1F *sec11_r0_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec11_R0");
    TH1F *sec11_r1_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec11_R1");
    TH1F *sec11_r2_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec11_R2");
    TH1F *sec21_r0_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec21_R0");
    TH1F *sec21_r1_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec21_R1");
    TH1F *sec21_r2_new = (TH1F*)qa_file_new->Get("h_TpcRawHitQA_adc_sec21_R2");
    
    TH1F *sec1_r0_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec1_R0");
    TH1F *sec1_r1_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec1_R1");
    TH1F *sec1_r2_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec1_R2");
    TH1F *sec11_r0_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec11_R0");
    TH1F *sec11_r1_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec11_R1");
    TH1F *sec11_r2_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec11_R2");
    TH1F *sec21_r0_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec21_R0");
    TH1F *sec21_r1_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec21_R1");
    TH1F *sec21_r2_ref = (TH1F*)qa_file_ref->Get("h_TpcRawHitQA_adc_sec21_R2");
    
    m_canvas->cd(1);
    DrawReference(sec1_r0_new, sec1_r0_ref);
    
    m_canvas->cd(2);
    DrawReference(sec1_r1_new, sec1_r1_ref);
    m_canvas->cd(3);
    DrawReference(sec1_r2_new, sec1_r2_ref);
    m_canvas->cd(4);
    DrawReference(sec11_r0_new, sec11_r0_ref);
    m_canvas->cd(5);
    DrawReference(sec11_r1_new, sec11_r1_ref);
    m_canvas->cd(6);
    DrawReference(sec11_r2_new, sec11_r2_ref);
    m_canvas->cd(7);
    DrawReference(sec21_r0_new, sec21_r0_ref);
    m_canvas->cd(8);
    DrawReference(sec21_r1_new, sec21_r1_ref);
    m_canvas->cd(9);
    DrawReference(sec21_r2_new, sec21_r2_ref);
    
    
    m_canvas->Update();





KSTestSummary::PushKSTest - Warning - received pValue = 0. Reset to an arbitary small non-zero value (e^(-15))


### Cluster Quantities

In [12]:
%%cpp




    TCanvas* m_canvas_chip = new TCanvas("m_canvas_chip", "TPC Plots", 1000, 800);
    m_canvas_chip->Divide(3, 3);
  

    
    // Get histograms
    TH1F *clusphisize_side0_r0_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_phisize_side0_0"));
    TH1F *clusphisize_side0_r1_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_phisize_side0_1"));
    TH1F *clusphisize_side0_r2_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_phisize_side0_2"));
    TH1F *clusphisize_side1_r0_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_phisize_side1_0"));
    TH1F *clusphisize_side1_r1_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_phisize_side1_1"));
    TH1F *clusphisize_side1_r2_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_phisize_side1_2"));
    TH1F *cluszsize_r0_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_zsize_0"));
    TH1F *cluszsize_r1_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_zsize_1"));
    TH1F *cluszsize_r2_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_zsize_2"));
    TH1F *clusrphierr_r0_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_rphi_error_0"));
    TH1F *clusrphierr_r1_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_rphi_error_1"));
    TH1F *clusrphierr_r2_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_rphi_error_2"));
    TH1F *cluszerr_r0_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_z_error_0"));
    TH1F *cluszerr_r1_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_z_error_1"));
    TH1F *cluszerr_r2_new = dynamic_cast<TH1F*>(qa_file_new->Get("h_TpcClusterQA_z_error_2"));
    
    TH1F *clusphisize_side0_r0_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_phisize_side0_0"));
    TH1F *clusphisize_side0_r1_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_phisize_side0_1"));
    TH1F *clusphisize_side0_r2_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_phisize_side0_2"));
    TH1F *clusphisize_side1_r0_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_phisize_side1_0"));
    TH1F *clusphisize_side1_r1_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_phisize_side1_1"));
    TH1F *clusphisize_side1_r2_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_phisize_side1_2"));
    TH1F *cluszsize_r0_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_zsize_0"));
    TH1F *cluszsize_r1_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_zsize_1"));
    TH1F *cluszsize_r2_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_zsize_2"));
    TH1F *clusrphierr_r0_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_rphi_error_0"));
    TH1F *clusrphierr_r1_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_rphi_error_1"));
    TH1F *clusrphierr_r2_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_rphi_error_2"));
    TH1F *cluszerr_r0_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_z_error_0"));
    TH1F *cluszerr_r1_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_z_error_1"));
    TH1F *cluszerr_r2_ref = dynamic_cast<TH1F*>(qa_file_ref->Get("h_TpcClusterQA_z_error_2"));
    
  

  
 
    std::cout << "DrawChipInfo completed" << std::endl;


DrawChipInfo completed


##### Cluster phi size South

In [13]:
    %%cpp
    TCanvas *canphisize = new TCanvas("canphisizesouth","TPC Cluster Phi Size",1000,800);
    canphisize->Divide(3,1);
    
    canphisize->cd(1);
    DrawReference(clusphisize_side0_r0_new, clusphisize_side0_r0_ref);
    canphisize->cd(2);
    DrawReference(clusphisize_side0_r1_new, clusphisize_side0_r1_ref);
    canphisize->cd(3);
    DrawReference(clusphisize_side0_r2_new, clusphisize_side0_r2_ref);
    
    canphisize->Update();

#### Cluster phi size North

In [14]:
    %%cpp
    TCanvas *canphisizen = new TCanvas("canphisizenorth","TPC Cluster Phi Size",1000,800);
    canphisizen->Divide(3,1);
    
    canphisizen->cd(1);
    DrawReference(clusphisize_side1_r0_new, clusphisize_side1_r0_ref);
    canphisizen->cd(2);
    DrawReference(clusphisize_side1_r1_new, clusphisize_side1_r1_ref);
    canphisizen->cd(3);
    DrawReference(clusphisize_side1_r2_new, clusphisize_side1_r2_ref);
    
    canphisizen->Update();

#### Cluster z size

In [15]:
    %%cpp
    TCanvas *canzsize = new TCanvas("canzsize","TPC Cluster Phi Size",1000,800);
    canzsize->Divide(3,1);
    
    canzsize->cd(1);
    DrawReference(cluszsize_r0_new, cluszsize_r0_ref);
    canzsize->cd(2);
    DrawReference(cluszsize_r1_new, cluszsize_r1_ref);
    canzsize->cd(3);
    DrawReference(cluszsize_r2_new, cluszsize_r2_ref);

    canzsize->Update();

### Cluster rphi error

In [16]:
    %%cpp
    TCanvas *rphierrcan = new TCanvas("rphierrcan","TPC Cluster RPhi Err",1000,800);
    rphierrcan->Divide(3,1);
    
    rphierrcan->cd(1);
    DrawReference(clusrphierr_r0_new, clusrphierr_r0_ref);
    rphierrcan->cd(2);
    DrawReference(clusrphierr_r1_new, clusrphierr_r1_ref);
    rphierrcan->cd(3);
    DrawReference(clusrphierr_r2_new, clusrphierr_r2_ref);

    rphierrcan->Update();

KSTestSummary::PushKSTest - Warning - received pValue = 0. Reset to an arbitary small non-zero value (e^(-15))


#### Cluster z error

In [17]:
  %%cpp
    TCanvas *zerrcan = new TCanvas("zerrcan","TPC Cluster z Err",1000,800);
    zerrcan->Divide(3,1);
    
    zerrcan->cd(1);
    DrawReference(cluszerr_r0_new, cluszerr_r0_ref);
    zerrcan->cd(2);
    DrawReference(cluszerr_r1_new, cluszerr_r1_ref);
    zerrcan->cd(3);
    DrawReference(cluszerr_r2_new, cluszerr_r2_ref);

    zerrcan->Update();

KSTestSummary::PushKSTest - Warning - received pValue = 0. Reset to an arbitary small non-zero value (e^(-15))


In [18]:
%%cpp

KSTestSummary::getInstance()->make_summary_txt("QA-tpc.txt");
KSTestSummary::getInstance()->make_summary_TCanvas()->Draw();

This notebook contains 24 KSTets: combined Chi2/nDoF = 667.174 / 48, and combined __p-Value = 5.98783e-110__
